<h1>Ufc punch counter version 2</h1>


<h3>This code maintains a seperate punch count for each fighter using the punch classification model trained in train.ipynb and another pose detection model called MoveNet for fighter classification</h3>

In [ ]:
# import all libraries

import cv2
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2 import model_zoo
from detectron2.data import DatasetCatalog,MetadataCatalog
import os

import imageio 
import matplotlib.pyplot as plt 
from IPython.display import HTML, display

# Calculations and Deep Learning library
import numpy as np 
import tensorflow as tf 
import tensorflow_hub as hub

In [ ]:
#import video and classification

vc = cv2.VideoCapture("mcgregorvsdiaz.mp4")
fourcc = cv2.VideoWriter_fourcc(*'XVID')
vw = cv2.VideoWriter("output4.avi",fourcc,20,(1280,720))

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.ROI_HEADS.NUM_CLASSES=3
cfg.OUTPUT_DIR = 'faster_rcnn_R_50_FPN_3x\\lr=0.001,itr=3000'
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
predictor = DefaultPredictor(cfg)


In [ ]:

# COLORS
cyan = (255, 255, 0)
magenta = (255, 0, 255)
EDGE_COLORS = {
    (0, 1): magenta,
    (0, 2): cyan,
    (1, 3): magenta,
    (2, 4): cyan,
    (0, 5): magenta,
    (0, 6): cyan,
    (5, 7): magenta,
    (7, 9): cyan,
    (6, 8): magenta,
    (8, 10): cyan,
    (5, 6): magenta,
    (5, 11): cyan,
    (6, 12): magenta,
    (11, 12): cyan,
    (11, 13): magenta,
    (13, 15): cyan,
    (12, 14): magenta,
    (14, 16): cyan
}

# MODEL
model = hub.load("https://tfhub.dev/google/movenet/multipose/lightning/1")
movenet = model.signatures["serving_default"]

# PARAMETERS
WIDTH = 256
HEIGHT = 128


# FUNCTION TO GET SHORTS COLOR
def getShortsColor(frame):
    threshold=0.11
    """Draws the keypoints on a image frame"""
    initial_shape = frame.shape
    image = cv2.resize(frame, (HEIGHT,WIDTH))
    # Resize to the target shape and cast to an int32 vector
    input_image = tf.cast(tf.image.resize_with_pad(image, WIDTH, HEIGHT), dtype=tf.int32)
    # Create a batch (input tensor)
    input_image = tf.expand_dims(input_image, axis=0)

    # Perform inference
    results = movenet(input_image)
    keypoints = results["output_0"].numpy()[:,:,:51].reshape((6,17,3))
    keypoints = keypoints[0]
    # Denormalize the coordinates : multiply the normalized coordinates by the input_size(width,height)
    denormalized_coordinates = np.squeeze(np.multiply(keypoints, [WIDTH,HEIGHT,1]))
    # Get the keypoint of the left thigh
    lowerBodyPoints = [denormalized_coordinates[11]]
    for keypoint in lowerBodyPoints:
        # Unpack the keypoint values : y, x, confidence score
        keypoint_y, keypoint_x, keypoint_confidence = keypoint


        if keypoint_confidence > threshold:
            """"
            Use keypoints to calculate top right and bottom left coordinates of region of interest
            Region of interest = portion of fighter's shorts to indetify color for distinction
            """
            topRight = (keypoint_x-3,keypoint_y-3)
            bottomLeft = (keypoint_x+3,keypoint_y+12)

    resultImage = frame[int(topRight[1]):int(bottomLeft[1]),int(topRight[0]):int(bottomLeft[0])]
    return resultImage

# FUNCTION TO COMPARE SHORT COLORS
    
def compareColors(base,shorts1):
    base = np.array(base)
    shorts1 = np.array(shorts1)

    diff1 = np.abs(base - shorts1)
    diff1 = np.mean(diff1.ravel())
    return diff1

    
# GET BASE SHORT COLORS - these will be compared with shorts colors of fighters in the video
# for classification purposes.
base1=[]
base2=[]

def getShortsColor(frame):
    threshold=0.1
    """Draws the keypoints on a image frame"""
    initial_shape = frame.shape
    image = cv2.resize(frame, (HEIGHT,WIDTH))
    # Resize to the target shape and cast to an int32 vector
    input_image = tf.cast(tf.image.resize_with_pad(image, WIDTH, HEIGHT), dtype=tf.int32)
    # Create a batch (input tensor)
    input_image = tf.expand_dims(input_image, axis=0)

    # Perform inference
    results = movenet(input_image)
    keypoints = results["output_0"].numpy()[:,:,:51].reshape((6,17,3))
    keypoints = keypoints[0]
    # Denormalize the coordinates : multiply the normalized coordinates by the input_size(width,height)
    denormalized_coordinates = np.squeeze(np.multiply(keypoints, [WIDTH,HEIGHT,1]))
    #Iterate through the points
    lowerBodyPoints = [denormalized_coordinates[11]]
    for keypoint in lowerBodyPoints:
        # Unpack the keypoint values : y, x, confidence score
        keypoint_y, keypoint_x, keypoint_confidence = keypoint
        
        topRight = (keypoint_x-3,keypoint_y-3)
        bottomLeft = (keypoint_x+3,keypoint_y+20)
    
    resultImage = image[int(topRight[1]):int(bottomLeft[1]),int(topRight[0]):int(bottomLeft[0])]

    if resultImage.shape!=(23, 6, 3): #the shape may not be equal if the point detected in near the edges of the image
        return np.ones((23, 6, 3)),[int(topRight[0]),int(topRight[1]),int(bottomLeft[0]),int(bottomLeft[1])]

    return resultImage,[int(topRight[0]),int(topRight[1]),int(bottomLeft[0]),int(bottomLeft[1])]


In [ ]:
# setting parameters for the classification

counter = 0
punches1=0
punches2=0
buffer=[]
fighterDetectionBuffer=[]

fighter1 = cv2.imread("fighters\\fighter1.png")
fighter2 = cv2.imread("fighters\\fighter2.png")

# these two shorts colors will be compared with frames in the video
base1,frame1 = getShortsColor(fighter1)
base2,frame2 = getShortsColor(fighter2)

plt.imshow(base2)
plt.show()

In [ ]:
from statistics import mode

while True:
    ret,img = vc.read()
    if not ret:
        break

    counter+=1
    # print(counter)

    # make the prediction
    outputs = predictor(img)

    #get  bounding box, labels and confidence levels
    boxes = outputs["instances"].pred_boxes.tensor.tolist()
    labels=outputs["instances"].pred_classes.tolist()
    conf_lvls=outputs["instances"].scores.tolist()

    #value format=(boxes,conf_lvl)
    punchBoxes=[]
    for i,box in enumerate(boxes):
        if labels[i]==2:
            punchBoxes.append([box,conf_lvls[i]])


    # add to punch count if punch is detected in 3 to 10 consecutive frames
    # this is because a punch may last several seconds depending on the fps of the video
    # I found it was 3-10 for this video, you can adjust it if not accurate
    if len(punchBoxes)>=1:         
        # if buffer is empty or has previous frame
        if len(buffer)==0 or buffer[len(buffer)-1]+1==counter:
            buffer.append(counter)
        # clear buffer if frame is not consecutive
        elif buffer[len(buffer)-1]+1!=counter:
            if len(buffer)>=3 and len(buffer)<=10:
                if mode(fighterDetectionBuffer)==1:
                    punches1+=1
                else:
                    punches2+=1

            buffer=[]
            fighterDetectionBuffer=[]
            buffer.append(counter)

        # classify punching fighter as figter 1 or fighter2 and increment the correspoding punch count:
        if len(buffer)==10:
            if len(buffer)>=3 and len(buffer)<=10:
                if mode(fighterDetectionBuffer)==1:
                    punches1+=1
                else:
                    punches2+=1

            buffer=[]
            fighterDetectionBuffer=[]
            buffer.append(counter)

        #maintain fightDetectionBuffer
        for box in punchBoxes:
            box = box[0] #remove confidence level from punchBoxes
            xmin,ymin,xmax,ymax = box
            xmin = int(xmin)
            ymin = int(ymin)
            xmax = int(xmax)
            ymax = int(ymax)

            fighter = img[ymin:ymax,xmin:xmax,::]
            color,shortsCoordinates = getShortsColor(img)

            # classify punching fighter as 1 or 2 by comparing their shorts colors with the base short colors
            # and choosing the one with the lesser difference
            diff1 = compareColors(base1,color)
            diff2 = compareColors(base2,color)

        if diff1<diff2:
            fighterDetectionBuffer.append(1)
        else:
            fighterDetectionBuffer.append(2)

        text = "Color difference of last puncher: {}(1) and {}(2) ".format(round(diff1),round(diff2))
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_size=1
        font_color = (0,255,0)
        font_thickness=2
        position = (50,150)

    #print stats for debug purposes
    print(counter,buffer,fighterDetectionBuffer)      

    
    # draw these boxes on the image
    for box in punchBoxes:
        cv2.rectangle(img,(int(box[0][0]),int(box[0][1])),(int(box[0][2]),int(box[0][3])),(255,0,0),2)

    cv2.putText(img,"Fighter 1: {}".format(punches1),(50,50),cv2.FONT_HERSHEY_SIMPLEX,2,(255,0,0),2)
    cv2.putText(img,"Fighter 2: {}".format(punches2),(50,100),cv2.FONT_HERSHEY_SIMPLEX,2,(255,0,0),2)

    # save the frames with detected punches for ebug purposes
    '''
    if len(punchBoxes)>0:
        cv2.imwrite("results\\{}.jpg".format(counter),img)
    '''

    vw.write(img)

vc.release()
vw.release()